#### Imports and Paths

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torchdata
import torchvision
import torchmetrics
import pytorch_lightning as pl

from tqdm import tqdm
from hyperparameters import Hyperparameters
from datasets import ResiscDataModule, viz_batch

%load_ext dotenv
%dotenv

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
DATA_ROOT = Path.home() / "datasets"

RESISC = DATA_ROOT / "resisc-45" 
UCMERCED = DATA_ROOT / "uc-merced" / "Images"
MLRSNET = DATA_ROOT / "mlrs-net"

In [3]:
### Set Random State


In [4]:
class MetricParser:

    clf_metrics: dict[str, callable] = {
        "accuracy": torchmetrics.Accuracy,
        "precision": torchmetrics.Precision,
        "recall": torchmetrics.Recall,
        "f1score": torchmetrics.F1Score,
    }

    reg_metrics: dict[str, callable] = {
        "mse": torchmetrics.MeanSquaredError,
        "mae": torchmetrics.MeanAbsoluteError
    }

    averaging: dict[str, str] = {
        "accuracy": "micro",
        "precision": "macro",
        "recall": "macro",
        "f1score": "macro"
    }

    def __init__(self, params: Hyperparameters):
        self.task = params.task
        self.num_classes = params.num_classes
        self.metric_names:list = params.metrics

    def parse(self):
        return [self.parse_metric(m) for m in self.metric_names]

    def parse_metric(self, metric_name) -> callable:
        t = self.task.split('-')[-1]

        if t == "classification":
            metric = self.clf_metrics[metric_name] 
            return metric(task = self.task.split('-')[0],
                          num_classes = self.num_classes,
                          average = self.averaging[metric_name])

        elif t == "regression":
            metric = self.reg_metrics[metric_name]
            return metric

#### Models

In [10]:
class ClassificationModel(pl.LightningModule):
    def __init__(self, model, params: Hyperparameters):
        super().__init__()
        self.model = model
        self.params = params
        self.model.classifier[-1] = torch.nn.Linear(
            in_features = self.model.classifier[-1].in_features,
            out_features = params.num_classes,
            bias = True
        )
        self._set_metrics()
        self.save_hyperparameters(
            {i:params.get_dict()[i] for i in params.get_dict().keys() if i!='criterion'},
            ignore = ["model"]
        ) 
    
    def forward(self, batch):
        x, _ = batch
        return self.model(x)

    def _set_metrics(self):
        metric_parser = MetricParser(self.params)
        self.test_metrics = torchmetrics.MetricCollection(
            metrics = metric_parser.parse(),
            prefix = "test_")
        
        self.val_metrics = self.test_metrics.clone(prefix = "val_")

        metric_parser.metric_names = ["accuracy"]
        self.train_metrics = torchmetrics.MetricCollection(
            metrics = metric_parser.parse(),
            prefix = "train_"
        )

    def training_step(self, batch, batch_idx):
        loss = self._forward_pass(batch, self.train_metrics)
        self.log("train_loss", loss, on_step = True, on_epoch = True)
        self.log_dict(self.train_metrics, on_epoch=True, on_step=False)
        return loss

    def test_step(self, batch, batch_idx):
        loss = self._forward_pass(batch, self.test_metrics)
        self.log("test_loss", loss)
        self.log_dict(self.test_metrics, on_epoch=True, on_step = False)
    
    def validation_step(self, batch, batch_idx):
        loss = self._forward_pass(batch, self.val_metrics)
        self.log("val_loss", loss)
        self.log_dict(self.val_metrics, on_epoch=True, on_step = False)
      
    def configure_optimizers(self):
        return self.params.optimizer(self.model.parameters(), 
                                     lr = self.params.learning_rate)

    def _forward_pass(self, batch, metrics):
        x, y = batch
        y_pred = self.model(x)
        print(y_pred.shape)
        print(y.shape)
        metrics(y_pred, y) 
        return self.params.criterion(y_pred, y)

In [11]:
experiment = Hyperparameters(
    task = "multiclass-classification",
    random_seed = 69,
    num_classes = 45,
    test_split = .25,
    metrics = ["accuracy", "f1score"],
    learning_rate = 1e-5,
    batch_size = 64,
    num_workers = 16,
    optimizer = torch.optim.Adam,
    criterion = torch.nn.CrossEntropyLoss(),
)
pl.seed_everything(69);

alexnet = torchvision.models.alexnet(weights = torchvision.models.AlexNet_Weights.DEFAULT)
resisc_dm = ResiscDataModule(RESISC, experiment)
classifier = ClassificationModel(alexnet, experiment)

Global seed set to 69


In [12]:
alexnet

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [13]:
trainer = pl.Trainer(fast_dev_run=True) 

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.


In [14]:
trainer.fit(model = classifier, datamodule = resisc_dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | model         | AlexNet          | 57.2 M
1 | test_metrics  | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | train_metrics | MetricCollection | 0     
---------------------------------------------------
57.2 M    Trainable params
0         Non-trainable params
57.2 M    Total params
228.753   Total estimated model params size (MB)
/home/sambhav/miniconda3/envs/dev/lib/python3.11/site-packages/pytorch_lightning/utilities/data.py:120: UserWarning: Your `IterableDataset` has `__len__` defined. In combination with multi-process data loading (when num_workers > 1), `__len__` could be inaccurate if each worker is not configured independently to avoid having duplicate data.
  rank_zero_warn(


Epoch 0:   0%|          | 0/1 [00:00<?, ?it/s] torch.Size([64, 45])
torch.Size([64])
Epoch 0: 100%|██████████| 1/1 [00:02<00:00,  2.04s/it]torch.Size([64, 45])
torch.Size([64])
Epoch 0: 100%|██████████| 1/1 [00:06<00:00,  6.40s/it]

`Trainer.fit` stopped: `max_steps=1` reached.


Epoch 0: 100%|██████████| 1/1 [00:06<00:00,  6.40s/it]


In [9]:
trainer.save_checkpoint("alexnet_train.ckpt")

In [10]:
trainer.test(model = classifier, datamodule = resisc_dm, ckpt_path = "alexnet_train.ckpt")

Restoring states from the checkpoint path at alexnet_train.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at alexnet_train.ckpt


Testing DataLoader 0: 100%|██████████| 99/99 [00:12<00:00,  7.69it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│  test_MulticlassAccuracy  │    0.1792929321527481     │
│  test_MulticlassF1Score   │    0.02902105078101158    │
│         test_loss         │     6.833883285522461     │
└───────────────────────────┴───────────────────────────┘

TEST Profiler Report

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Action                                                                                                                                                         	|  Mean duration (s)	|  Num calls      	|  Total time (s) 	|  Percentage %   	|
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Total                                                                                                                                                          	|  -              	|  18947          	

[{'test_loss': 6.833883285522461,
  'test_MulticlassAccuracy': 0.1792929321527481,
  'test_MulticlassF1Score': 0.02902105078101158}]

In [18]:
resisc_dm.setup("test")
preds = trainer.predict(model = classifier, dataloaders = resisc_dm.test_dataloader())
truths = next(iter(resisc_dm.test_dataloader()))[1]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 99/99 [00:12<00:00,  7.95it/s]


In [20]:
confm = torchmetrics.classification.MulticlassConfusionMatrix(num_classes=experiment.num_classes)
confm.update(preds[0], truths)

In [ ]:
!nvidia-smi